In [20]:
%load_ext autoreload
%autoreload 2
from factor_processing_utils import *

In [ ]:
start_date = "2015-01-01"
end_date = "2025-07-01"
index_item = "000852.XSHG"
change_day = 20
month_day = 20
year_day = 252

stock_universe = INDEX_FIX(start_date, end_date, index_item)
stock_list = stock_universe.columns.tolist()
date_list = stock_universe.index.tolist()


In [44]:
factor_definition = Factor("cash_flow_from_operating_activities_ttm_0") / Factor(
    "total_assets_ttm_0"
)
factor_name = "cfoa_ttm"
raw_path = f"factor_lib/raw/{factor_name}_{index_item}_{start_date}_{end_date}.pkl"
neu_path = f"factor_lib/neu/{factor_name}_{index_item}_{start_date}_{end_date}.pkl"

In [ ]:
raw_factor = execute_factor(factor_definition, stock_list, start_date, end_date)
factor_clean = data_clean(raw_factor, stock_universe, index_item)
# 计算IC
ic, performance = calc_ic(factor_clean, change_day, index_item, factor_name)

In [ ]:
# 保存因子
raw_factor.to_pickle(raw_path)
factor_clean.to_pickle(neu_path)
# 分组回测
group_return, turnover = group_g(
    factor_clean, change_day, 10, index_item, name=factor_name
)